In [2]:
import Load_CIFAR10
import numpy as  np
from sklearn import preprocessing

In [6]:
from imp import reload 
reload(Load_CIFAR10) 

<module 'Load_CIFAR10' from 'E:\\JupyterEnviroment\\Learning\\MachineLearning\\Network\\Load_CIFAR10.py'>

In [22]:
trainX,trainY,testX,testY = Load_CIFAR10.load_Data_Lables()
#拉成一维向量
trainX = trainX.reshape(trainX.shape[0],-1)
testX = testX.reshape(testX.shape[0],-1)
#标准化
scaler = preprocessing.StandardScaler().fit(trainX)
trainX = scaler.transform(trainX)
testX = scaler.transform(testX)
#给每个数据都要多加一维，作为偏置位
train_bias = np.ones([trainX.shape[0],1],float)
test_bias = np.ones([testX.shape[0],1],float)
trainX = np.c_[trainX,train_bias]
testX = np.c_[testX,test_bias]

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [4]:
class NeuralNetwork:
    def __init__(self,layers,alpha):
        '''初始函数
        layers:tuple类型,每一个元素表示该层的单元个数
        alpha:学习率
        '''
        self.layers = layers
        self.W = []
        self.dropOut = [None for i in range(len(layers)-1)]
         
        self.alpha = alpha
        self.out = [None for i in range(len(layers)-1)]
        self.result = [None for i in range(len(layers))]#每一层运算后的结果
        
        for i in range(len(layers)-2):
            #输入层和输出层不需要权重
            #要考虑偏置值
            w = np.random.randn(layers[i]+1,layers[i+1]+1)
            self.W.append(w)
        
        #最后一层输出层不需要加偏置项
        w = np.random.randn(layers[-2]+1,layers[-1])
        self.W.append(w)
        
        #转换一下类型
        self.W = np.array(self.W)
        self.result = np.array(self.result)

        
    def Relu(self,value):
        '''激活函数采用整流线性单元，但是为了避免出现梯度饱和，采用渗漏型整流线性单元
        value:上一层的输出入与隐藏层的权重乘积的结果
        '''
        v = []
        for i in value:
            d = [j if j>=0 else j*0.01 for j in i]
            v.append(d)
        return np.array(v)

    def softMax(self,value):
        max_dict = [ [np.max(i)] for i in value]
        x = np.exp((value-max_dict))
        
        return np.array([ i/np.sum(i) for i in x])
    
    def fit(self,trainX,trainY,batch_size,reg,num):
        num_train = trainX.shape[0]
        for i in range(num):
            #进行小批量训练，随机抽取一批数量为batch_size的数据集进行训练
            batch_index = np.random.choice(num_train,batch_size)
            batch_X = trainX[batch_index,:]
            batch_Y = trainY[batch_index]
            self.train(batch_X,batch_Y,0.2)
            if i%100==0 :
#                 print('W',self.W[-1])
                print('loss',self.calculateLoss(trainX,trainY))
    def getdropOutVector(self,size):
        '''获取一次用于dropOut的向量Load_CIFAR10.py
        size:表示几行几列
        '''
        _sum = 0
        while _sum<=0:
            dropOutV = np.random.rand(size[1])
            dropOutV = dropOutV>0.5
            _sum = sum(dropOutV)
            
        return dropOutV
    
    def predict(self,X):
        out = X
        #计算隐藏层
        for i in range(1,len(self.layers)-1):
            #上一层的输出与当前层的权重相乘
            net = out.dot(self.W[i-1])*0.5
            
            #经过激活函数
            out = self.Relu(net)
        
        #计算输出层
        net = out.dot(self.W[-1])
        out = self.softMax(net)
        return out
    
    def calculateLoss(self,trainX,trainY):
        predictions = self.predict(trainX)
        
        loss = sum(np.sum(trainY*np.log(predictions),1))
        
        return loss
    def train(self,batch_X,batch_Y,reg):
        self.result[0] = batch_X
        
        #计算隐藏层
        for i in range(1,len(self.layers)-1):
            
            #上一层的输出与当前层的权重相乘
            net = self.result[i-1].dot(self.W[i-1])
            scaler = preprocessing.StandardScaler().fit(net)
            net = scaler.transform(net)
            
            #生成当前层dropout向量
            #保存到self.dropOut中去，用于反向传播
            #再把Dropout向量乘上当前的输出
            dropOutVec = self.getdropOutVector([1,net.shape[1]])
            self.dropOut[i-1] = dropOutVec
            net = net*dropOutVec
            
            #经过激活函数
            out = self.Relu(net)
            self.out[i-1] = out
            self.result[i] = out

        #计算输出层
        net = self.result[-2].dot(self.W[-1])#因为数据没有归一化或者标准化化？导致数据过大，爆了。。
        out = self.softMax(net)
        self.result[-1] = out
        #反向传播
        #先是输出层的反向传播梯度更新
        df_dout = out-batch_Y
        
        dout_dnet = []
        for i in range(batch_X.shape[0]):
            _dout_dnet = [(np.sum(self.result[-1][i])-self.result[-1][i][j])*self.result[-1][i][j]/(np.sum(self.result[-1][i])**2) for j in range(self.layers[-1])]
            dout_dnet.append(_dout_dnet)
        dout_dnet = np.array(dout_dnet)

        D = [None for i in range(len(self.W))]#保存 df/dout *dout/dnet
        D[-1] = df_dout*dout_dnet

        #接着是隐藏层的
        for i in range(len(self.W)-2,-1,-1):
            dout_dnet = []
            for j in self.out[i]:
                _dout_dnet = [1 if m>=0 else 0.01 for m in j]
                dout_dnet.append(_dout_dnet)
                
            dout_dnet = np.array(dout_dnet)
    
            D[i] = D[i+1].dot(self.W[i+1].T)*dout_dnet#注意点乘还是*乘
        
        
        #为输入出增加一层全都是1的dropout层，输入层是不需要dropout的，为了公式一致所以加上一层都是为1的dropout层
        last_dropoutVector = np.array([1.0 for i in range(self.W[-1].shape[-1])])
        self.dropOut[-1] = last_dropoutVector
          
        #然后是梯度更新
        for i in range(len(self.W)):
            self.W[i] = self.W[i] - self.alpha*(self.W[i]*reg+1/batch_X.shape[0]*self.result[i].T.dot(D[i]*self.dropOut[i]))#注意点乘还是*乘
        
        

In [5]:
from sklearn.datasets import load_digits #手写数字数据集
from sklearn.preprocessing import LabelBinarizer #标签二值化处理
from sklearn.model_selection import train_test_split #训练和测试集分隔
# 载入数据
digits = load_digits()

# 输入的数据
X = digits.data
X = X.reshape([X.shape[0],-1])
# 标签数据
T = digits.target
# 数据切分，默认测试集占0.25
X_train,X_test,y_train,y_test = train_test_split(X,T)
# (1797,) [0 1]
labels_train = LabelBinarizer().fit_transform(y_train)

In [44]:
bp_net = NeuralNetwork([X_train.shape[1]-1,16,10],0.1)
bp_net.fit(trainX=X_train,trainY=labels_train,batch_size=256,num=100,reg=0.9)

loss 141762.907799532


In [6]:
from sklearn.metrics import classification_report, confusion_matrix
output = bp_net.predict(X_test)
predictions = np.argmax(output,axis=1)
print(classification_report(predictions,y_test))
#这里有一个问题，如果训练的次数少了，会导致有很少部分的数字会出现一次都不会被检测到

NameError: name 'bp_net' is not defined

In [7]:
#失败。
bp_net = NeuralNetwork([trainX.shape[1]-1,128,64,32,16,10],1e-3)
bp_net.fit(trainX=trainX[:100],trainY=trainY[:100],batch_size=64,num=5000,reg=0.9)
bp_net.predict(trainX[:100],trainY[:100])

NameError: name 'trainX' is not defined

In [4]:
#原版数据载入
# # 载入数据
# digits = load_digits()
# print(digits.images.shape) #结果：(1797, 8, 8)

# # 输入的数据
# X = digits.data
# X = X.reshape([X.shape[0],-1])
# # 标签数据
# T = digits.target
# print(X.shape)
# print(T.shape)
# # print(X.shape, X[:2], '\n')
# # print(T.shape, T[:2])


(1797, 8, 8)
(1797, 64)
(1797,)


In [19]:
import numpy as np
from sklearn.datasets import load_digits #手写数字数据集
from sklearn.preprocessing import LabelBinarizer #标签二值化处理
from sklearn.model_selection import train_test_split #训练和测试集分隔
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# 载入数据
digits = load_digits()
print(digits.images.shape) #结果：(1797, 8, 8)

# 输入的数据
X = digits.data
# 标签数据
T = digits.target
print(X.shape, X[:2], '\n')
print(T.shape, T[:2])

# (1797,) [0 1]

# 定义一个神经网络 64-100-10，隐藏层神经元单元为100,输出层神经元单元为10
# 输入层至隐藏层的权值矩阵
V = np.random.random([64,100])*2 -1
# 隐藏层至输出层的权值矩阵
W = np.random.random([100,10])*2 -1

# 数据切分，默认测试集占0.25
X_train,X_test,y_train,y_test = train_test_split(X,T)

# 标签二值化，独热编码
# 1 -> 0100000000
labels_train = LabelBinarizer().fit_transform(y_train)
print(y_train[:2])
print(labels_train[:2])
#打印结果：
#[5 9]
#[[0 0 0 0 0 1 0 0 0 0]
 #[0 0 0 0 0 0 0 0 0 1]]
 
def sigmoid(x):
    return 1/(1+np.exp(-x))

# 激活函数的导数
def dsigmoid(x):
    return x*(1-x)

#预测值计算
def predict(x):
    L1 = sigmoid(np.dot(x,V))
    L2 = sigmoid(np.dot(L1,W))
    return L2

#模型训练
def train(X, T, steps=10000, lr=0.11):
    global V,W
    for n in range(steps + 1):
        #从样本中随机选取一个数据
        i = np.random.randint(X.shape[0])
        x = X[i]
        x = np.atleast_2d(x) #转换为2D矩阵
        #BP算法公式
        L1 = sigmoid(np.dot(x,V))
        L2 = sigmoid(np.dot(L1,W))
        
        L2_delta = (T[i] - L2)*dsigmoid(L2)
        L1_delta = L2_delta.dot(W.T)*dsigmoid(L1)
        
        W += lr * L1.T.dot(L2_delta)
        V += lr * x.T.dot(L1_delta)
        
        if n%1000 == 0:
            output = predict(X_test)
            predictions = np.argmax(output, axis=1)
            acc = np.mean(np.equal(predictions, y_test))
            print('iter: ' + str(n + 1) + " acc: " + str(acc))

train(X_train,labels_train,10000)
#输出结果：
# iter: 1 acc: 0.8644444444444445
# iter: 1001 acc: 0.8733333333333333
# iter: 2001 acc: 0.8688888888888889
# ...
# iter: 5001 acc: 0.8711111111111111
# ...
# iter: 22001 acc: 0.8666666666666667
# iter: 23001 acc: 0.8711111111111111
#....
# iter: 28001 acc: 0.8711111111111111
# iter: 29001 acc: 0.8666666666666667
# iter: 30001 acc: 0.8711111111111111

# 结果评估
output = predict(X_test)
predictions = np.argmax(output,axis=1)
print(predictions)
print(classification_report(predictions,y_test))


(1797, 8, 8)
(1797, 64) [[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.]] 

(1797,) [0 1]
[4 1]
[[0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]]
iter: 1 acc: 0.10888888888888888
iter: 1001 acc: 0.5177777777777778
iter: 2001 acc: 0.6088888888888889
iter: 3001 acc: 0.7888888888888889
iter: 4001 acc: 0.8533333333333334
iter: 5001 acc: 0.8933333333333333
iter: 6001 acc: 0.9066666666666666
iter: 7001 acc: 0.9377777777777778
iter: 8001 acc: 0.9377777777777778
iter: 9001 acc: 0.9444444444444444
iter: 10001 acc: 0.933333333

In [7]:
print(predictions.shape)
print(y_test.shape)

(450,)
(450,)
